This contains non-COI tips currently (which is fine) but would need to filter them out

In [11]:
import csv
import dendropy
from collections import defaultdict
from collections import Counter
import numpy as np


In [12]:
countries = ["cuba", "haiti", "puerto_rico", "dominican_republic", "jamaica", "other_caribbean"]

In [13]:
#call on root
def parse_introductions(node, country_of_interest, subtree_dict, node_to_ancestor):
    
    if node.parent_node in subtree_dict.keys(): #if the parent is the root of the subtree, add myself
        subtree_dict[node.parent_node].add(node)
        node_to_ancestor[node] = node.parent_node    
        
    elif node.parent_node in node_to_ancestor: #if the parent is already in a subtree, add myself
        ancestral_node = node_to_ancestor[node.parent_node]
        subtree_dict[ancestral_node].add(node)
        node_to_ancestor[node] = ancestral_node
        
    else: #not in a subtree
        if node.annotations["location"].value in country_of_interest: #am I the root of a subtree?
            subtree_dict[node] = set()
        
    if node.taxon:
        pass
    else:
        for child in node.child_node_iter():
            parse_introductions(child, country_of_interest, subtree_dict, node_to_ancestor)

    return subtree_dict, node_to_ancestor
    

## running

In [14]:
for sero in ["DENV1","DENV2", "DENV3", "DENV4"]:

    tree = dendropy.Tree.get(path=f"{sero}_dta.mcc", schema = "nexus")
    for country in countries:
        subtree_dict = defaultdict(set)
        node_to_ancestor = {}

        country_lst = [country]
        subtree_dict, node_to_ancestor = parse_introductions(tree.seed_node, country_lst, subtree_dict,  node_to_ancestor)

        with open(f"{sero}_{country}.csv", 'w') as fw:
            fieldnames = ["height", "source", "size_in_country", "tips_country", "tips_all"]
            writer = csv.DictWriter(fw, fieldnames=fieldnames)
            writer.writeheader()
            for node, tips in subtree_dict.items():

                tip_country = []
                tip_all = []

                if node.taxon:
                    tip_all.append(node.taxon.label)
                    tip_country.append(node.taxon.label)

                for tip in tips:
                    if tip.taxon:
                        tip_all.append(tip.taxon.label)
                        if tip.annotations['location'].value == country:
                            tip_country.append(tip.taxon.label)

                write_dict = {}
                
                write_dict["source"] = node.parent_node.annotations['location'].value
                write_dict["height"] = node.annotations['height'].value
                write_dict["size_in_country"] = len(tip_country)
                write_dict["tips_country"] = ";".join(tip_country)
                write_dict["tips_all"] = ";".join(tip_all)

                writer.writerow(write_dict)


    subtree_dict = defaultdict(set)
    node_to_ancestor = {}

    subtree_dict, node_to_ancestor = parse_introductions(tree.seed_node,countries, subtree_dict,  node_to_ancestor)

    with open(f"{sero}_whole_caribbean.csv", 'w') as fw:
        fieldnames = ["height", "source", "destination", "size_in_country", "tips_country", "tips_all"]
        writer = csv.DictWriter(fw, fieldnames=fieldnames)
        writer.writeheader()
        for node, tips in subtree_dict.items():

            tip_country = []
            tip_all = []

            if node.taxon:
                tip_all.append(node.taxon.label)
                tip_country.append(node.taxon.label)

            for tip in tips:
                if tip.taxon:
                    tip_all.append(tip.taxon.label)
                    if tip.annotations['location'].value in countries:
                        tip_country.append(tip.taxon.label)

            write_dict = {}

            write_dict["source"] = node.parent_node.annotations['location'].value
            write_dict["destination"] = node.annotations['location'].value
            write_dict["height"] = node.annotations['height'].value
            write_dict["size_in_country"] = len(tip_country)
            write_dict["tips_country"] = ";".join(tip_country)
            write_dict["tips_all"] = ";".join(tip_all)

            writer.writerow(write_dict)